In [1]:
%matplotlib qt5

import hyperspy.api as hs
import numpy as np

import h5py
import matplotlib.pyplot as plt

import glob



import sklearn as skl
from sklearn import preprocessing
from sklearn.decomposition import PCA, NMF, FactorAnalysis
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.pipeline import Pipeline

import seaborn as sns
import pandas as pd


from skcmeans.algorithms import Probabilistic
from skcmeans.algorithms import Probabilistic, Possibilistic, GustafsonKesselMixin

# Convert the hdf5 data from Sigray HHDF5 standard to Hyperspy standard

In [ ]:
!cd 'data/ISE_500sqaures_A21-016_Map1_001_rawmap/'
!ls 'data/ISE_500sqaures_A21-016_Map1_002_rawmap/'

In [2]:
def h5printR(item, leading = ''):
    for key in item:
        if isinstance(item[key], h5py.Dataset):
            print(leading + key + ': ' + str(item[key].shape))
        else:
            print(leading + key)
            h5printR(item[key], leading + '  ')

# Print structure of a `.h5` file            
def h5print(filename):
    with h5py.File(filename, 'r') as h:
        print(filename)
        h5printR(h, '  ')

In [ ]:
xrf_data = h5py.File('data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0001','r' )

In [ ]:
h5print('data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0001')

In [ ]:
row_1=xrf_data['entry']['data']['data']

In [ ]:
row_1[0].shape

#### Print the attributes associated with the data. 

these are needed to scale the data correctly etc.

In [ ]:
for att in xrf_data['entry']['data']['data'].attrs:
    print(att,xrf_data['entry']['data']['data'].attrs[att])

# Physical paramters 

need these t ocorrectly scale the pixels and arrange the data.

In [ ]:
pix_x, iterations, num_ch =xrf_data['entry']['data']['data'].shape

print(pix_x, iterations, num_ch)

In [ ]:
#pix_x_size=xrf_data['entry']['data']['data'].attrs['ScanXWidth']/pix_x

#pix_y_size=np.abs(xrf_data['entry']['data']['data'].attrs['ScanXWidth'].attrs['ScanYWidth'])/(corr_data['entry']['detector']['data1'].attrs['ScanYNumberOfPoints']-1)
#print(pix_x_size,pix_y_size)

In [ ]:
xrf_map = hs.signals.Signal1D(xrf_data['entry']['data']['data'])
xrf_map.plot()

In [ ]:
xrf_map

In [ ]:
xrf_map=xrf_map.sum(axis=0)

In [ ]:
xrf_map

In [ ]:
xrf_map.plot()

# this is the file conversion

In [ ]:
xrf_map = hs.signals.Signal1D(xrf_data['entry']['data']['data'])
xrf_map=xrf_map.sum(axis=0)
print(xrf_map)
print(xrf_map.data.shape)

In [11]:


save_path='/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sq_A21-016_map2_002_hspy/'

#this changes the sigray file to a hyperspy... proably should delete these once we have stacked.
for fname in glob.glob('/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.*'):

    print(fname)
    map=int(fname.split('xsp3.')[1])
    xrf_data = h5py.File(fname,'r' )
    xrf_map = hs.signals.Signal1D(xrf_data['entry']['data']['data'])
#sum the 'y- axis' as realised we ahave two passes per pixel, so intigrate    
    xrf_map=xrf_map.sum(axis=0)
    xrf_map.crop(axis=0,start=1)
    if (map%2) == 0: xrf_map.data=np.flip(xrf_map.data, axis=0)
    
    #xrf_map.axes_manager[0].name = 'X'
    #xrf_map.axes_manager['X'].units = 'mm'
    #xrf_map.axes_manager['X'].scale = pix_x_size
    #xrf_map.axes_manager['X'].offset = corr_data['entry']['detector']['data1'].attrs['MotorXLineStartRBV']

    #xrf_map.axes_manager[1].name = 'Y'
    #xrf_map.axes_manager['Y'].units = 'mm'
    #xrf_map.axes_manager['Y'].scale = pix_y_size
    #xrf_map.axes_manager['Y'].offset = corr_data['entry']['detector']['data1'].attrs['MotorYLineStartRBV']


    #xrf_map.axes_manager[1].name= 'Energy'
    #xrf_map.axes_manager['Energy'].units = 'kev'
    #xrf_map.axes_manager['Energy'].scale = corr_data['entry']['detector']['data1'].attrs['ECAL'][1]
    #xrf_map.axes_manager['Energy'].offset = corr_data['entry']['detector']['data1'].attrs['ECAL'][0]

    #print(xrf_map.axes_manager)
 
    
    print(map)
    xrf_map.save(save_path+'at16_map2_002_{:05d}'.format(map))
#################################################################################
    
xrf_stack=hs.load(save_path+'at16_map2_002_*.hspy',stack=True)

#xrf_stack.axes_manager[1].name = 'Y'
#xrf_stack.axes_manager['Y'].units = 'mm'
#xrf_stack.axes_manager['Y'].scale = pix_y_size
#xrf_stack.axes_manager['Y'].offset = corr_data['entry']['detector']['data1'].attrs['MotorYLineStartRBV']

print(xrf_stack.axes_manager)

#be good to have dialoge etc for the file name but this works....

xrf_stack.save(save_path+'at16_map2_002_mapped')

xrf_stack.plot()

/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0123
123
/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0089
89
/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0047
47
/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0096
96
/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0032
32
/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0022
22
/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0088
88
/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0139
139
/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0063
63
/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0028
2

34
/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0120
120
/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0030
30
/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0091
91
/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0117
117
/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0029
29
/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0142
142
/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0156
156
/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0041
41
/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0094
94
/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0

166
/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0161
161
/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0065
65
/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0048
48
/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0083
83
/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0185
185
/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0051
51
/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0122
122
/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0146
146
/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.0071
71
/media/2TB_NVMe_2/science_2/2022_XRF_melts/data/ISE_500sqaures_A21-016_Map2_002_rawmap/xsp3.

## Quick data inspection.

ran PCA to see what the data looks like

In [ ]:
xrf_stack=hs.load(save_path+'at16_map2_002_*.hspy',stack=True)

#xrf_stack.axes_manager[1].name = 'Y'
#xrf_stack.axes_manager['Y'].units = 'mm'
#xrf_stack.axes_manager['Y'].scale = pix_y_size
#xrf_stack.axes_manager['Y'].offset = corr_data['entry']['detector']['data1'].attrs['MotorYLineStartRBV']

print(xrf_stack.axes_manager)

#be good to have dialoge etc for the file name but this works....
xrf_stack.change_dtype('float32')

xrf_stack.save(save_path+'at16_map2_002_mapped')

In [6]:
xrf_stack

<Signal1D, title: ISE_500sq_A21-016_map2_002_hspy, dimensions: (202, 201|4096)>

In [ ]:
xrf_stack.plot()

In [12]:
xrf_stack.change_dtype('float32')


xrf_stack.save(save_path+'at16_map2_002_mapped_crop')

In [15]:
plt.close('all')

In [13]:

xrf_stack.decomposition(normalize_poissonian_noise=True, algorithm='SVD', output_dimension=20)

xrf_stack.plot_explained_variance_ratio(log=True, vline=True)

xrf_stack.plot_decomposition_results()

Decomposition info:
  normalize_poissonian_noise=True
  algorithm=SVD
  output_dimension=20
  centre=None


In [14]:
xrf_stack.plot_cumulative_explained_variance_ratio()

<AxesSubplot:xlabel='Principal component', ylabel='Cumulative explained variance ratio'>

**note** iterative cropping above showed that most data/ analysis was better when cropping down to the shape (157, 63|2001)

In [ ]:
xrf_stack.save(save_path+'lisheen_low_res_map_data_Crop')

## re-explore

get 6 good factors... need to label peaks etc but starting to pull out data.



In [16]:
xrf_stack.decomposition(normalize_poissonian_noise=True, algorithm='NMF', output_dimension=6)


xrf_stack.plot_decomposition_results()

/home/jeinsle/anaconda3/envs/mla/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "
/home/jeinsle/anaconda3/envs/mla/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


Decomposition info:
  normalize_poissonian_noise=True
  algorithm=NMF
  output_dimension=6
  centre=None
scikit-learn estimator:
NMF(n_components=6)


In [ ]:
xrf_stack.decomposition(normalize_poissonian_noise=True, algorithm='NMF', output_dimension=5)


xrf_stack.plot_decomposition_results()

## examine with factor analysis

In [17]:
pipeline = Pipeline([("FA", FactorAnalysis(n_components=10,rotation = 'varimax'))])

xrf_stack.decomposition(normalize_poissonian_noise=True, algorithm=pipeline, return_info=True,output_dimension=11)

xrf_stack.plot_decomposition_results()

Decomposition info:
  normalize_poissonian_noise=True
  algorithm=Pipeline(steps=[('FA', FactorAnalysis(n_components=10, rotation='varimax'))])
  output_dimension=11
  centre=None
scikit-learn estimator:
Pipeline(steps=[('FA', FactorAnalysis(n_components=10, rotation='varimax'))])


In [18]:
xrf_stack.plot_decomposition_results()

In [ ]:
fa_load= xrf_stack.get_decomposition_loadings()
#fa_load.save('snv_eds_analysis/plage_TD_eds_FA_4_load')

fa_fact= xrf_stack.get_decomposition_factors()
#fa_fact.set_elements(elements)


#fa_fact.save('snv_eds_analysis/plage_TD_eds_FA_4_fact')

In [ ]:
fa_facts, ydim,xdim=fa_load.data.shape

fact_load_vect= pd.DataFrame((fa_load.data.reshape(fa_facts, ydim*xdim).T), columns = ['Factor 1','Factor 2','Factor 3','Factor 4', 'Factor 5'])


In [ ]:
fact_load_vect

In [ ]:
fa_facts, ydim,xdim=fa_load.data.shape
fa_vect=fa_load.data.reshape(fa_facts,ydim*xdim).transpose()
fa_vect.shape

In [ ]:
loadings_to_cluster=fa_vect[:,:fa_facts]
loadings_to_cluster.shape

In [ ]:
class PGK(Probabilistic, GustafsonKesselMixin):
    pass

In [ ]:
num_clus=10
pgk = PGK(n_clusters =num_clus, n_init=10).fit(loadings_to_cluster)
# Process results for visualisation
print(pgk.memberships_)
labels_ = np.argmax(pgk.memberships_, axis=1)
memberships_ = pgk.memberships_[range(len(pgk.memberships_)), labels_] 

labels = labels_.reshape([ydim,xdim])
labels=hs.signals.Signal2D(labels)
labels.plot(cmap='tab10')

mems=pgk.memberships_.reshape(ydim,xdim,num_clus)
print(mems.shape)

mem_maps=hs.signals.Signal2D(mems)
mem_maps=mem_maps.transpose(signal_axes=(2,0))
mem_maps.change_dtype('float32')
mem_maps.plot(cmap='viridis')

In [ ]:
plt.close('all')

mem_maps.plot(cmap='viridis')

## next steps

1. makes sense to calcualte an elbow test or examine the BIC to determine the 'optimal' number of clusters
2. explore applying HDBSCAN on the FA/ PCA factors
3. would also to explore the UMAP pretreatment then HDBSCAN
4. maybe blind source separation (ie ICA) after PCA.

for 2 and 3 need a different virtual enviroment.

## actually lets try ICA

In [ ]:
xrf_stack.decomposition(normalize_poissonian_noise=True, algorithm='SVD', output_dimension=7)


In [ ]:
xrf_stack.blind_source_separation(number_of_components=7)

xrf_stack.plot_bss_results()